In [3]:
%pip install docling --extra-index-url https://download.pytorch.org/whl/cpu
%pip install docling-core python-dotenv langchain-text-splitters pymupdf4llm pymupdf ragas
%pip install -qU langsmith langchain-core langchain-community langchain-openai langchain-qdrant langchain_experimental 

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import warnings

warnings.filterwarnings(action="ignore", category=UserWarning, module="pydantic|torch")
warnings.filterwarnings(action="ignore", category=FutureWarning, module="easyocr")

In [5]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "PDF Loader Comparison"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")



In [6]:
FILE_PATH = "https://arxiv.org/pdf/2206.01062"
# FILE_PATH="https://s2.q4cdn.com/470004039/files/doc_earnings/2024/q3/filing/_10-Q-Q3-2024-As-Filed.pdf"

In [ ]:
# from langchain_community.tools.arxiv.tool import ArxivQueryRun, arxiv
# from langchain_community.tools.arxiv import arxiv
# arxiv_query = ArxivQueryRun()
# arxiv_paper = arxiv_query.invoke(FILE_PATH)
# arxiv_paper

# Generate Synthetic Dataset

## Prepare

In [7]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.testset.docstore import Document, DocumentStore,InMemoryDocumentStore
from langchain_experimental.text_splitter import SemanticChunker

llm_as_generator = "gpt-4o"
generator_llm = ChatOpenAI(model=llm_as_generator)
llm_as_critic = "gpt-4o"
critic_llm = ChatOpenAI(model=llm_as_critic)
embedding_model_for_generator = "text-embedding-3-large"
embedding_model_dimensions = 3072
embeddings = OpenAIEmbeddings(model=embedding_model_for_generator,dimensions=embedding_model_dimensions)

# splitter = SemanticChunker(embeddings)
# docstore = InMemoryDocumentStore(splitter,)

generator = TestsetGenerator.from_langchain( # Default uses TokenTextSplitter
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.4,
    multi_context: 0.4,
    reasoning: 0.2
}

### Generate Synthetic testset using raga
We are going to use `PyMuPDFLoader` to read a pdf file for generating question. In the next iteration we will try and use advanced pdf loader.

In [8]:
from langchain_community.document_loaders import PyMuPDFLoader

documents = PyMuPDFLoader(file_path=FILE_PATH).load()

In [9]:
title = str(documents[0].metadata.get("title"))
documents

[Document(metadata={'source': 'https://arxiv.org/pdf/2206.01062', 'file_path': 'https://arxiv.org/pdf/2206.01062', 'page': 0, 'total_pages': 9, 'format': 'PDF 1.5', 'title': 'DocLayNet: A Large Human-Annotated Dataset for Document-Layout Analysis', 'author': 'Birgit Pfitzmann, Christoph Auer, Michele Dolfi, Ahmed S. Nassar, and Peter Staar', 'subject': '-  Information systems  ->  Document structure; -  Applied computing  ->  Document analysis; -  Computing methodologies  ->  Machine learning; Computer vision; Object detection; ', 'keywords': 'PDF document conversion, layout segmentation, object-detection, data set, Machine Learning', 'creator': 'LaTeX with acmart 2018/04/14 v1.53 Typesetting articles for the Association for Computing Machinery and hyperref 2020-05-15 v7.00e Hypertext links for LaTeX', 'producer': 'pdfTeX-1.40.21', 'creationDate': 'D:20220603003241Z', 'modDate': 'D:20220603003241Z', 'trapped': ''}, page_content="DocLayNet: A Large Human-Annotated Dataset for\nDocument-

In [8]:
testset = generator.generate_with_langchain_docs(documents, 20, distributions, with_debugging_logs=True) # Default  RunConfig(max_retries=15, max_wait=90)

embedding nodes:   0%|          | 0/42 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/20 [00:00<?, ?it/s]

[ragas.testset.filters.DEBUG] context scoring: {'clarity': 2, 'depth': 3, 'structure': 2, 'relevance': 3, 'score': 2.5}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['Impact of Document Split', 'DocLayNet', 'Model performance', 'Random page-wise splitting', 'Dataset Comparison']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 2, 'depth': 3, 'structure': 2, 'relevance': 3, 'score': 2.5}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['DocLayNet dataset', 'Prediction Performance', 'Mask R-CNN R50 network', 'Document layout analysis', 'Human annotation']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 2, 'depth': 3, 'structure': 2, 'relevance': 3, 'score': 2.5}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['DocLayNet dataset', 'Prediction Performance', 'Mask R-CNN R50 network', 'Document layout analysis', 'Human annotation']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 2, 'depth': 3, 'structure': 2, 'relevance'

In [9]:
testset.to_pandas()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How does random page-wise splitting affect mod...,"[-average improves\nby around 5%, in particula...",Random page-wise splitting of DocLayNet can ea...,simple,[{'source': 'https://arxiv.org/pdf/2206.01062'...,True
1,What advancements does the DocLayNet dataset b...,"[KDD ’22, August 14–18, 2022, Washington, DC, ...",The DocLayNet dataset provides the document co...,simple,[{'source': 'https://arxiv.org/pdf/2206.01062'...,True
2,What is the significance of DocLayNet in the f...,[DocLayNet: A Large Human-Annotated Dataset fo...,The answer to given question is not present in...,simple,[{'source': 'https://arxiv.org/pdf/2206.01062'...,True
3,What characteristics make technical content do...,[DocLayNet: A Large Human-Annotated Dataset fo...,Technical content documents are challenging to...,simple,[{'source': 'https://arxiv.org/pdf/2206.01062'...,True
4,What is the prediction performance (mAP@0.5-0....,"[KDD ’22, August 14–18, 2022, Washington, DC, ...",The prediction performance (mAP@0.5-0.95) of d...,simple,[{'source': 'https://arxiv.org/pdf/2206.01062'...,True
5,What measures were implemented to ensure clear...,"[KDD ’22, August 14–18, 2022, Washington, DC, ...",To ensure clear and unbiased baseline numbers ...,simple,[{'source': 'https://arxiv.org/pdf/2206.01062'...,True
6,How does DocLayNet's wider variety of document...,"[-average improves\nby around 5%, in particula...",DocLayNet's wider variety of document layouts ...,simple,[{'source': 'https://arxiv.org/pdf/2206.01062'...,True
7,How many total annotation instances are provid...,"[test-split. Last\nbut not least, we compare t...",The DocLayNet dataset provides 91104 total ann...,simple,[{'source': 'https://arxiv.org/pdf/2206.01062'...,True
8,What doc categories and annotation challenges ...,"[KDD ’22, August 14–18, 2022, Washington, DC, ...",The document categories tackled at KDD 2022 in...,multi_context,[{'source': 'https://arxiv.org/pdf/2206.01062'...,True
9,"How do Mask R-CNN, Faster R-CNN, and YOLOv5x6 ...",[\nsize of the DocLayNet dataset with similar ...,Mask R-CNN and Faster R-CNN produce very compa...,multi_context,[{'source': 'https://arxiv.org/pdf/2206.01062'...,True


In [26]:
from langsmith import Client

client = Client()

dataset_name = "DocLayNet7"

# dataset = client.create_dataset(
#     dataset_name=dataset_name,
#     description=title
# )
# for test in testset.to_pandas().iterrows():
#   client.create_example(
#       inputs={
#           "question": test[1]["question"]
#       },
#       outputs={
#           "answer": test[1]["ground_truth"]
#       },
#       metadata={
#           "context": test[0]
#       },
#       dataset_id=dataset.id
#   )

  # What if the dataset already exists or dont want to rerun this again?
# dataset1 = client.read_dataset(dataset_name)


# RAG Pipelines

## PDF Loaders

In [10]:
from pdfloader import PDFLoader

pymupdf_loader = PDFLoader(FILE_PATH,PDFLoader.LoaderType.PYMUPDF)
pymupdf4llm_loader = PDFLoader(FILE_PATH,PDFLoader.LoaderType.PYMUPDF4LLM)
docling_loader = PDFLoader(FILE_PATH,PDFLoader.LoaderType.DOCLING)


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

## Text Cunking

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size=1000
chunk_overlap=50
chunking_strategy = "RecursiveCharacterTextSplitter"

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

In [12]:
pymupdf_documents = pymupdf_loader.load()
pymupdf4llm_documents = pymupdf4llm_loader.load()
docling_documents = docling_loader.load()

pymupdf_chunks = text_splitter.split_documents(pymupdf_documents)
pymupdf4llm_chunks = text_splitter.split_documents(pymupdf4llm_documents)
docling_chunks = text_splitter.split_documents(docling_documents)


Loading 1 documents with LoaderType.PYMUPDF loader...
Loading 1 documents with LoaderType.PYMUPDF4LLM loader...
Loading PDF from /var/folders/fy/bc970z1n26gflr1xr_4kr5380000gn/T/tmp4f_or245.pdf...
Processing /var/folders/fy/bc970z1n26gflr1xr_4kr5380000gn/T/tmp4f_or245.pdf...
[                                        ] (0/9===[====                                    ] (1/9===[========                                ] (2/9====[=============                           ] (3/===[=================                       ] (4/====[======================                  ] (5/9===[==========================              ] (6/9====[===============================         ] (7/===[===================================     ] (8/====[========================================] (9/9]
Loading 1 documents with LoaderType.DOCLING loader...


In [13]:
from langchain_openai import OpenAIEmbeddings

embedding_modle_for_rag_chain = "text-embedding-3-large"
embeddings = OpenAIEmbeddings(model=embedding_modle_for_rag_chain)

In [14]:
from langchain_community.vectorstores import Qdrant

pymupdf_collection_name = "pymupdf_collection"
pymupdf_vectorstore = Qdrant.from_documents(
    documents=pymupdf_chunks,
    embedding=embeddings,
    location=":memory:",
    collection_name=pymupdf_collection_name
)
pymupdf_retriever = pymupdf_vectorstore.as_retriever()

pymupdf4llm_collection_name = "pymupdf4llm_collection"
pymupdf4llm_vectorstore = Qdrant.from_documents(
    documents=pymupdf4llm_chunks,
    embedding=embeddings,
    location=":memory:",
    collection_name=pymupdf4llm_collection_name
)
pymupdf4llm_retriever = pymupdf4llm_vectorstore.as_retriever()


docling_collection_name = "docling_collection"
docling_vectorstore = Qdrant.from_documents(
    documents=docling_chunks,
    embedding=embeddings,
    location=":memory:",
    collection_name=docling_collection_name
)
docling_retriever = docling_vectorstore.as_retriever()



In [15]:
docling_retriever.invoke("BARSK")

[Document(metadata={'dl_doc_hash': '5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c846634ff924e635e0dc', '_id': '9c36425fbc0b4b8581a2678710eb702f', '_collection_name': 'docling_collection'}, page_content='Articles category. We also avoided class labels that are tightly linked to the semantics of the text. Labels such as Author and Affiliation, as seen in DocBank, are often only distinguishable by discriminating on'),
 Document(metadata={'dl_doc_hash': '5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c846634ff924e635e0dc', '_id': '98e9cbd286cd4e4092131a212b1be5e2', '_collection_name': 'docling_collection'}, page_content='were carried out over a timeframe of 12 weeks, after which 8 of the 40 initially allocated annotators did not pass the bar.'),
 Document(metadata={'dl_doc_hash': '5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c846634ff924e635e0dc', '_id': 'c694ac5b90a749c7bb1a9858855c9aee', '_collection_name': 'docling_collection'}, page_content='out-performs humans on selected labels such as T

In [16]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [17]:
from langchain_openai import ChatOpenAI

llm_model_for_rag = "gpt-4o-mini"
llm = ChatOpenAI(model=llm_model_for_rag)

In [18]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

pymupdf_rag_chain = (
    {"context": itemgetter("question") | pymupdf_retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

pymupdf4llm_rag_chain = (
    {"context": itemgetter("question") | pymupdf4llm_retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

docling_rag_chain = (
    {"context": itemgetter("question") | docling_retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)




In [19]:
pymupdf_rag_chain.invoke({"question" : "What benefits does DocLayNet provide for ML in layout analysis vs. other datasets?"})

'DocLayNet provides several benefits for machine learning in layout analysis compared to other datasets like PubLayNet and DocBank. Firstly, it offers a wide variability in layouts, as it contains 80,863 manually annotated pages from diverse data sources, which helps improve the accuracy of layout predictions on different document types. Additionally, DocLayNet is human-annotated, which can enhance the quality of the annotations. It also includes a subset of double- and triple-annotated pages to assess inter-annotator agreement, allowing for better evaluation of the annotation quality. Lastly, it is formatted in COCO format, which is commonly accepted in the community, making it easier to use with existing machine learning frameworks.'

In [20]:
pymupdf4llm_rag_chain.invoke({"question" : "What benefits does DocLayNet provide for ML in layout analysis vs. other datasets?"})


'DocLayNet provides several benefits for machine learning in layout analysis compared to other datasets:\n\n1. **Human Annotation**: Unlike PubLayNet and DocBank, DocLayNet relies on human annotation to generate the dataset, which ensures reliable layout ground-truth.\n\n2. **Large Layout Variability**: It includes diverse and complex layouts from a wide variety of public sources, adding significant value to the dataset.\n\n3. **Detailed Label Set**: DocLayNet defines 11 class labels for distinguishing layout features in high detail, which is more detailed compared to the 5 labels in PubLayNet.\n\n4. **Redundant Annotations**: A fraction of the pages in DocLayNet carries more than one human annotation, enabling experimentation with annotation uncertainty and quality control analysis.\n\nThese aspects contribute to improving and fine-tuning novel machine learning methods for document layout analysis.'

In [21]:
docling_rag_chain.invoke({"question" : "What benefits does DocLayNet provide for ML in layout analysis vs. other datasets?"})


'DocLayNet provides several benefits for machine learning in layout analysis compared to other datasets:\n\n1. **Human Annotation**: Unlike PubLayNet and DocBank, DocLayNet relies on human annotation instead of automated approaches, which can lead to higher quality data.\n\n2. **Large Layout Variability**: The dataset includes diverse and complex layouts from a wide variety of public sources, addressing the lack of layout variability found in other datasets that are sourced primarily from scientific article repositories.\n\n3. **Detailed Label Set**: DocLayNet defines 11 class labels to distinguish layout features in high detail, which is more detailed than the 5 labels provided by PubLayNet and offers a different focus than the 13 labels in DocBank.\n\n4. **Inter-Annotator Agreement**: DocLayNet includes a subset of double- and triple-annotated pages to determine inter-annotator agreement, providing a measure of reliability for the annotations.\n\n5. **Robust Model Performance**: Mode

# Langsmith evaluation

In [22]:
llm_as_judge = "gpt-4o"
eval_llm = ChatOpenAI(model=llm_as_judge)

In [23]:
from langsmith.evaluation import LangChainStringEvaluator

qa_evaluator = LangChainStringEvaluator("qa", config={"llm": eval_llm}) 
cot_qa_evaluator = LangChainStringEvaluator("cot_qa",config={"llm": eval_llm})
context_qa_evaluator = LangChainStringEvaluator("context_qa", config={"llm": eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        }
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    },
)

scoring_accuracy_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "criteria": {
            "accuracy": "Score 1: Completely inaccurate\nScore 5: Somewhat accurate\nScore 10: Completely accurate"
        },
        "normalize_by": 10,
        "llm": eval_llm,
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    },
)



In [34]:
from langsmith.evaluation import evaluate
from langsmith import Client

os.environ["LANGCHAIN_TRACING_V2"] = "true"

client = Client(auto_batch_tracing=False)
dist =str({simple: 0.4,multi_context: 0.4,reasoning: 0.2})
evaluate(
    pymupdf_rag_chain.invoke,
    data=dataset_name,
    experiment_prefix="pymupdf",
    max_concurrency=2,
    client=client,
    evaluators=[
        qa_evaluator,
        cot_qa_evaluator,
        context_qa_evaluator,
        labeled_helpfulness_evaluator,
        scoring_accuracy_evaluator
    ],
    metadata={
            "chain": "pymupdf_rag_chain",
            "dataset": dataset_name,
            "title": title,
            "generator_model": llm_as_generator,
            "critic_model": llm_as_critic,
            "embedding_model_generator": embedding_model_for_generator,
            "embedding_model_dimensions": str(embedding_model_dimensions),
            "distributions": dist,
            "chunk_size": str(chunk_size),
            "chunk_overlap": str(chunk_overlap),
            "chunking_strategy": chunking_strategy,
            "embedding_model_rag_chain": embedding_modle_for_rag_chain,
            "llm_model_rag": llm_model_for_rag,
            "llm_as_judge": llm_as_judge            
        },
)


View the evaluation results for experiment: 'pymupdf-8a8b9611' at:
https://smith.langchain.com/o/e319c8f1-73bd-5d44-8897-d19a191ebc54/datasets/c9a1d7bd-5aab-4a5a-be54-b2d049f5c0aa/compare?selectedSessions=24840705-4f1f-41a3-a474-f59d8c51bd06




0it [00:00, ?it/s]

<ExperimentResults pymupdf-8a8b9611>

In [35]:
evaluate(
    pymupdf4llm_rag_chain.invoke,
    data=dataset_name,
    experiment_prefix="pymupdf4llm",
    max_concurrency=2,
    client=client,
    evaluators=[
        qa_evaluator,
        cot_qa_evaluator,
        context_qa_evaluator,
        labeled_helpfulness_evaluator,
        scoring_accuracy_evaluator
    ],
    metadata={
            "chain": "pymupdf4llm_rag_chain",
            "dataset": dataset_name,
            "title": title,
            "generator_model": llm_as_generator,
            "critic_model": llm_as_critic,
            "embedding_model_generator": embedding_model_for_generator,
            "embedding_model_dimensions": str(embedding_model_dimensions),
            "distributions": dist,
            "chunk_size": str(chunk_size),
            "chunk_overlap": str(chunk_overlap),
            "chunking_strategy": chunking_strategy,
            "embedding_model_rag_chain": embedding_modle_for_rag_chain,
            "llm_model_rag": llm_model_for_rag,
            "llm_as_judge": llm_as_judge            
        },
)

View the evaluation results for experiment: 'pymupdf4llm-6547e321' at:
https://smith.langchain.com/o/e319c8f1-73bd-5d44-8897-d19a191ebc54/datasets/c9a1d7bd-5aab-4a5a-be54-b2d049f5c0aa/compare?selectedSessions=2ad051e3-f354-4e7d-b213-005e9e6ed52e




0it [00:00, ?it/s]

<ExperimentResults pymupdf4llm-6547e321>

In [36]:
evaluate(
    docling_rag_chain.invoke,
    data=dataset_name,
    experiment_prefix="docling",
    max_concurrency=2,
    client=client,
    evaluators=[
        qa_evaluator,
        cot_qa_evaluator,
        context_qa_evaluator,
        labeled_helpfulness_evaluator,
        scoring_accuracy_evaluator
    ],
    metadata={
            "chain": "docling_rag_chain",
            "dataset": dataset_name,
            "title": title,
            "generator_model": llm_as_generator,
            "critic_model": llm_as_critic,
            "embedding_model_generator": embedding_model_for_generator,
            "embedding_model_dimensions": str(embedding_model_dimensions),
            "distributions": dist,
            "chunk_size": str(chunk_size),
            "chunk_overlap": str(chunk_overlap),
            "chunking_strategy": chunking_strategy,
            "embedding_model_rag_chain": embedding_modle_for_rag_chain,
            "llm_model_rag": llm_model_for_rag,
            "llm_as_judge": llm_as_judge            
        },
)

View the evaluation results for experiment: 'docling-9db64e5f' at:
https://smith.langchain.com/o/e319c8f1-73bd-5d44-8897-d19a191ebc54/datasets/c9a1d7bd-5aab-4a5a-be54-b2d049f5c0aa/compare?selectedSessions=efcb1c8c-ae18-4e90-8fd9-239a1b89fc1f




0it [00:00, ?it/s]

<ExperimentResults docling-9db64e5f>

In [42]:
dist =str({simple: 0.4,multi_context: 0.4,reasoning: 0.2})

print("dataset_name: " + dataset_name)
print("title: "+ title)
print("generator_model: "+ llm_as_generator)
print("critic_model: "+ llm_as_critic)
print("embedding_model_generator: "+ embedding_model_for_generator)
print("embedding_model_dimensions: "+ str(embedding_model_dimensions))
print("chunk_size:"+ str(chunk_size))
print("chunk_overlap:"+ str(chunk_overlap))
print("chunking_strategy: "+ chunking_strategy)
print("embedding_model_rag_chain: "+ embedding_modle_for_rag_chain)
print("llm_model_rag: "+ llm_model_for_rag)
print("llm_as_judge: "+ llm_as_judge)

dataset_name: DocLayNet7
title: DocLayNet: A Large Human-Annotated Dataset for Document-Layout Analysis
generator_model: gpt-4o
critic_model: gpt-4o
embedding_model_generator: text-embedding-3-large
embedding_model_dimensions: 3072
chunk_size:1000
chunk_overlap:50
chunking_strategy: RecursiveCharacterTextSplitter
embedding_model_rag_chain: text-embedding-3-large
llm_model_rag: gpt-4o-mini
llm_as_judge: gpt-4o
